In [1]:
import vrep 
import sys
import time 
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

from tank import *

In [2]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

Connected to remote API server


In [3]:
proximity_sensors=["EN","ES","NE","NW","SE","SW","WN","WS"]
proximity_sensors_handles=[0]*8

# get handle to proximity sensors
for i in range(len(proximity_sensors)):
    err_code,proximity_sensors_handles[i] = vrep.simxGetObjectHandle(clientID,"Proximity_sensor_"+proximity_sensors[i], vrep.simx_opmode_blocking)
    
#read and print values from proximity sensors
#first reading should be done with simx_opmode_streaming, further with simx_opmode_buffer parameter
for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_streaming)

In [4]:
def search_for_para_place(search_time = 10):
    tank.forward(5)
    t = time.time()
    # condition = True
    values = []
    while True:
        
        condition = True
        if (time.time()-t)>search_time:
            tank.stop()
            print("Didn't found parking spot ")
            return 0
        important_list = ['EN',"ES","NE","SE"]
        values = []
        for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
            
            if sensor_name in important_list:
                
                err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
                if(err_code == 0):
                    values.append(np.linalg.norm(detectedPoint))
        if len(values) == 4:
            
            for value in values:
                if ((value<1.3)|(value>1.85)):   # if any condition not in range we will be out of range
                    condition = False

            # if we reached that point that means that all value are in defined range = we found parking spot
            
            if condition:
                tank.stop()
                print("Found parking spot")
                return 1 
    

                
                    
                        

In [5]:
# now after we found parking spot, we need to position out tank correctly.
# to do that we will focus on SE sensor that should represent distance from 'car behind'
# around 3 we want tank to stop 
def prepare_parking_logic():
    # for now let's ignore spped 
    
    # rules
    in_se = ctrl.Antecedent(np.arange(1.3,3.5,0.05),'in_se')
    out_velo = ctrl.Consequent(np.arange(0,5.1,0.1),'out_velo')
    out_velo.defuzzify_method = 'centroid'
    in_se['low'] = fuzz.trimf(in_se.universe,[1.3,1.3,2.2])
    in_se['medium'] = fuzz.trimf(in_se.universe,[1.3,2.2,3.5])
    in_se['high'] = fuzz.trimf(in_se.universe,[2.2,3.5,3.5])
    out_velo['low'] = fuzz.trimf(out_velo.universe,[0,0,2.1])

    out_velo['medium'] = fuzz.trimf(out_velo.universe,[0,2.1,2.5])

    out_velo['high'] = fuzz.trimf(out_velo.universe,[2.5,5,5])
    
    acc_rule = ctrl.Rule(antecedent= in_se['low'] ,consequent = out_velo['high'])

    rul0 = ctrl.Rule(antecedent=(in_se['medium'] ),consequent=out_velo['medium'])

    dec_rule = ctrl.Rule(antecedent=in_se['high'],consequent=out_velo['low'])
    speed_ctrl = ctrl.ControlSystem([dec_rule,acc_rule,rul0])
    velo_updater = ctrl.ControlSystemSimulation(speed_ctrl)
    
    
    
    
    return velo_updater

In [6]:
step1 = prepare_parking_logic()

In [7]:
def prepare_parking(step1 = step1):
    sensor = "SE"
    counter = 0
    keep_track_of_previous_value = 0
    while True:
        
        for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
            if sensor_name ==sensor:
                err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
                
                if(err_code == 0): 
                    distance_data = np.linalg.norm(detectedPoint)
                    if distance_data > keep_track_of_previous_value:  
                        # sensor czasami zaczyna zwracać irracjonalnie niskie wartości, pomimo że na symulacji widac, ze jeszcze nie łapie kolejnego auta więc spodziewamy się co raz większych odległości
                        counter +=1
                        step1.input['in_se'] = distance_data
                        step1.compute()
                        tank.forward(step1.output['out_velo'])
                        keep_track_of_previous_value = distance_data
                        if counter % 10 == 0:
                            print("Proximity_sensor",distance_data,"Velocity",tank.rightvelocity)
                        
        if tank.rightvelocity < 0.95:
            print("Velocity below 0.95 stoping the tank; Tank prepared to begin parking")
            tank.stop()
            return 1
                        
                        
                    

In [8]:
def rotate_tank():
    sensor = 'NE'
    tank.backward(1)
    tank.turn_left()
    t = time.time()
    while True:
        
        for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
            if sensor_name == sensor:
                err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
                
                if(err_code == 0): 
                    distance_data = np.linalg.norm(detectedPoint)
                    # print(distance_data)
                    if time.time() - t >2.85:
                        tank.stop()
                        return 1
                        

In [9]:

def prepare_backing_logic():
    # for now let's ignore spped 
    
    # rules
    in_sw = ctrl.Antecedent(np.arange(1.0,3,0.05),'in_sw')
    out_velo = ctrl.Consequent(np.arange(0,1,0.01),'out_velo')
    out_velo.defuzzify_method = 'centroid'
    in_sw['low'] = fuzz.trimf(in_sw.universe,[1.0,1.0,1.6])
    in_sw['medium'] = fuzz.trimf(in_sw.universe,[1.6,2.2,3])
    in_sw['high'] = fuzz.trimf(in_sw.universe,[2.2,3,3])
    out_velo['low'] = fuzz.trimf(out_velo.universe,[0,0,0.4])

    out_velo['medium'] = fuzz.trimf(out_velo.universe,[0,0.4,0.7])

    out_velo['high'] = fuzz.trimf(out_velo.universe,[0.7,1,1])
    
    acc_rule = ctrl.Rule(antecedent= in_sw['low'] ,consequent = out_velo['low'])

    rul0 = ctrl.Rule(antecedent=(in_sw['medium'] ),consequent=out_velo['medium'])

    dec_rule = ctrl.Rule(antecedent=in_sw['high'],consequent=out_velo['high'])
    speed_ctrl = ctrl.ControlSystem([dec_rule,acc_rule,rul0])
    velo_updater = ctrl.ControlSystemSimulation(speed_ctrl)
    
    
    
    
    return velo_updater

In [10]:
step2= prepare_backing_logic()

In [11]:
def go_back(step1 = step2):
    sensor = "SE"
    tank.stop()
    tank.backward(0)
    tank.backward(3.3)
    t = time.time()
    while True:
        
        for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
            if sensor_name ==sensor:
                err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer )
                
                if(err_code == 0): 
                    distance_data = np.linalg.norm(detectedPoint)
                    timer = time.time()
                    if timer > t + 1:  
                        # sensor czasami zaczyna zwracać irracjonalnie niskie wartości, pomimo że na symulacji widac, ze jeszcze nie łapie kolejnego auta więc spodziewamy się co raz większych odległości
                        
                        step1.input['in_sw'] = distance_data
                        step1.compute()
                        # - + + i
                        if tank.rightvelocity + step1.output['out_velo'] >0:
                            tank.stop()
                            return 1 
                        tank.backward(-tank.rightvelocity - step1.output['out_velo'])
                        
                        
                        print("Proximity_sensor",distance_data,"Velocity",tank.rightvelocity)
                        t = timer
                        
       
                        

In [12]:
def correct():
    tank.turn_right()
    t = time.time()
    tank.forward(0)
    while True:
        t1 = time.time()
        if t1 - t >2:
            tank.stop()
            return
        
        tank.turn_right()
        

In [13]:
def parking_combined():
    place = search_for_para_place()
    if place == 1:
        prepare_parking()
        rotate_tank()
        go_back()
        correct()
        return 1
    else:
        print("Place not found")
        return 0 
    
    # time.sleep(10)
    # vrep.simxStopSimulation(clientID,vrep.simx_opmode_oneshot) # stop the simulation in vrep     
    

In [14]:
parking_combined()

Found parking spot
Proximity_sensor 1.450768036427388 Velocity 3.4679265852152215
Proximity_sensor 1.562032228174243 Velocity 3.145172183269877
Proximity_sensor 1.6754443778328711 Velocity 2.885874210451782
Proximity_sensor 1.756117887788756 Velocity 2.7211112364335115
Proximity_sensor 1.8428652497379174 Velocity 2.5484864414966077
Proximity_sensor 1.91558398134129 Velocity 2.3987006118518646
Proximity_sensor 2.0057042490375174 Velocity 2.1929550924793793
Proximity_sensor 2.0621642795984982 Velocity 2.0431399849439664
Proximity_sensor 2.121523054757822 Velocity 1.8566087008450978
Proximity_sensor 2.5496896977450545 Velocity 1.419921059197179
Proximity_sensor 2.8571858975299698 Velocity 1.194661701412309
Proximity_sensor 3.183698637305863 Velocity 0.9151515844784411
Velocity below 0.95 stoping the tank; Tank prepared to begin parking
Proximity_sensor 0.6837961837411602 Velocity -3.1666666666666665
Proximity_sensor 1.8917654837496514 Velocity -2.805811550706843
Proximity_sensor 1.2154067

1